# PART I

In [ ]:
# Q1: Some of the facilities charge a fee to members, but some do not.
# Write a SQL query to produce a list of the names of the facilities that do.

In [ ]:
SELECT name
FROM Facilities
WHERE membercost > 0

In [ ]:
# Q2: How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT(name)
FROM Facilities
WHERE membercost = 0

In [ ]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question

In [ ]:
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0
AND membercost < 0.20 * monthlymaintenance

In [ ]:
# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.

In [ ]:
SELECT *
FROM Facilities
WHERE name LIKE '% 2'

In [ ]:
# Q5: Produce a list of facilities, with each labelled as
# 'cheap' or 'expensive', depending on if their monthly maintenance cost is
# more than $100. Return the name and monthly maintenance of the facilities
# in question.

In [ ]:
SELECT name,
CASE WHEN monthlymaintenance >100
THEN 'expensive'
WHEN monthlymaintenance <100
THEN 'cheap'
ELSE 'neither'
END AS Costs
FROM Facilities

In [ ]:
# Q6: You'd like to get the first and last name of the last member(s)
# who signed up. Try not to use the LIMIT clause for your solution.

In [ ]:
SELECT firstname, surname
FROM Members
ORDER BY joindate DESC

In [ ]:
# Q7: Produce a list of all members who have used a tennis court.
# Include in your output the name of the court, and the name of the member
# formatted as a single column. Ensure no duplicate data, and order by
# the member name.

In [ ]:
SELECT DISTINCT m.memid, m.surname, m.firstname
FROM Bookings AS b
INNER JOIN Facilities AS f
    ON b.facid = f.facid
INNER JOIN Members AS m
    ON m.memid = b.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY m.surname

In [ ]:
# Q8: Produce a list of bookings on the day of 2012-09-14 which
# will cost the member (or guest) more than $30. Remember that guests have
# different costs to members (the listed costs are per half-hour 'slot'), and
# the guest user's ID is always 0. Include in your output the name of the
# facility, the name of the member formatted as a single column, and the cost.
# Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT 
    f.name,
    CONCAT( m.surname, ', ', m.firstname ) AS 'member name',
    CASE WHEN m.memid <>0
    THEN f.membercost * b.slots
    ELSE f.guestcost * b.slots END AS cost
FROM Bookings AS b
INNER JOIN Members AS m
    ON m.memid = b.memid
INNER JOIN Facilities AS f
    ON f.facid = b.facid
WHERE starttime LIKE '2012-09-14%'
ORDER BY cost DESC
LIMIT 12

In [ ]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
SELECT 
    f.name,
    CONCAT( m.surname, ', ', m.firstname ) AS 'member name',
    (SELECT CASE WHEN m.memid <>0
    THEN f.membercost * b.slots
    ELSE f.guestcost * b.slots END) AS cost
FROM Bookings AS b
INNER JOIN Members AS m
    ON m.memid = b.memid
INNER JOIN Facilities AS f
    ON f.facid = b.facid
WHERE starttime LIKE '2012-09-14%'
ORDER BY cost DESC
LIMIT 12

# PART II

In [20]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
engine = create_engine('sqlite:///')

In [ ]:
table_names = engine.table_names()
print(table_names)

In [ ]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! 

In [ ]:
con = engine.connect()
rs = con.execute('SELECT name, revenue FROM Facilities AS f WHERE revenue < 1000 SORT BY revenue')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

In [ ]:
# Alternative option
SELECT DISTINCT f.name, (

SELECT
CASE WHEN m.memid <>0
THEN f.membercost * b.slots
WHEN m.memid =0
THEN f.guestcost * b.slots
END
) AS revenue
FROM Facilities AS f
INNER JOIN Bookings AS b ON f.facid = b.facid
INNER JOIN Members AS m ON b.memid = m.memid
ORDER BY f.name

In [ ]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [ ]:
con = engine.connect()
rs = con.execute("SELECT CONCAT( m.surname, ', ', m.firstname ) AS members, memid, recommendedby FROM Members AS m ORDER BY members")
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

In [ ]:
SELECT CONCAT(m.surname, ', ', m.firstname) AS members, memid, recommendedby 
FROM Members AS m 
ORDER BY members

In [ ]:
# Q12: Find the facilities with their usage by member, but not guests

In [ ]:
con = engine.connect()
rs = con.execute("SELECT CONCAT( m.surname, ', ', m.firstname ) AS members, f.name FROM Bookings AS b INNER JOIN Facilities AS f ON f.facid = b.facid INNER JOIN Members AS m ON b.memid = m.memid WHERE m.memid <>0")
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()

In [ ]:
SELECT CONCAT( m.surname, ', ', m.firstname ) AS members, f.name
FROM Bookings AS b
INNER JOIN Facilities AS f ON f.facid = b.facid
INNER JOIN Members AS m ON b.memid = m.memid
WHERE m.memid <>0

In [ ]:
# Q13: Find the facilities usage by month, but not guests

In [ ]:
con = engine.connect()
rs = con.execute("SELECT DISTINCT f.name, EXTRACT(MONTH FROM b.starttime) AS months, COUNT( b.memid ) AS usage_by_month FROM Bookings AS b INNER JOIN Facilities AS f ON b.facid = f.facid WHERE b.memid <>0 GROUP BY f.name, months ORDER BY usage_by_month DESC")
df.columns = rs.keys()
con.close()

In [ ]:
SELECT 
    DISTINCT f.name, 
    EXTRACT(MONTH FROM b.starttime) AS months, COUNT( b.memid ) AS usage_by_month
FROM Bookings AS b
INNER JOIN Facilities AS f 
    ON b.facid = f.facid
WHERE b.memid <>0
GROUP BY f.name, months
ORDER BY usage_by_month DESC